In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = tf.keras.utils.image_dataset_from_directory('Train_', image_size=(224,224), shuffle=True, batch_size=128)

Found 2100 files belonging to 5 classes.


In [4]:
data.class_names

['Ak_', 'Ala_Idris_', 'Buzgulu_', 'Dimnit_', 'Nazli_']

In [5]:
dataset = data.map(lambda x, y: (x/255, y))

In [6]:
len(dataset)

17

In [7]:
train_size = 10
valid_size = 3
test_size = 4


In [8]:
train = dataset.take(train_size)
val = dataset.skip(train_size).take(valid_size)
test = dataset.skip(train_size+valid_size).take(test_size)

In [9]:
from tensorflow.keras.layers import Conv2D , MaxPooling2D , InputLayer,Flatten , GlobalAveragePooling2D ,AveragePooling2D, Dense , Dropout ,Activation , BatchNormalization, RandomFlip,  RandomRotation
from tensorflow.keras.models import Sequential

In [11]:
import tensorflow as tf

def dense_block(x, blocks, name):
    for i in range(blocks):
        x = conv_block(x, 32, name=name + '_block' + str(i + 1))
    return x

def transition_block(x, reduction, name):
    bn_axis = 3 if tf.keras.backend.image_data_format() == 'channels_last' else 1
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_bn')(x)
    x = tf.keras.layers.Activation('relu', name=name + '_relu')(x)
    x = tf.keras.layers.Conv2D(int(tf.keras.backend.int_shape(x)[bn_axis] * reduction), 1, use_bias=False,
                name=name + '_conv')(x)
    x = tf.keras.layers.AveragePooling2D(2, strides=2, name=name + '_pool')(x)
    return x

def conv_block(x, growth_rate, name):
    bn_axis = 3 if tf.keras.backend.image_data_format() == 'channels_last' else 1
    x1 = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                    epsilon=1.001e-5,
                                    name=name + '_0_bn')(x)
    x1 = tf.keras.layers.Activation('relu', name=name + '_0_relu')(x1)
    x1 = tf.keras.layers.Conv2D(4 * growth_rate, 1,
                        use_bias=False,
                        name=name + '_1_conv')(x1)
    x1 = tf.keras.layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(x1)
    x1 = tf.keras.layers.Activation('relu', name=name + '_1_relu')(x1)
    x1 = tf.keras.layers.Conv2D(growth_rate, 3,
                        padding='same',
                        use_bias=False,
                        name=name + '_2_conv')(x1)
    x = tf.keras.layers.Concatenate(axis=bn_axis, name=name + '_concat')([x, x1])
    return x

def DenseNet(blocks, input_shape=None, classes=1000):
    img_input = tf.keras.layers.Input(shape=input_shape)

    bn_axis = 3 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    x = tf.keras.layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
    x = tf.keras.layers.Conv2D(64, 7, strides=2, use_bias=False, name='conv1_conv')(x)
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='conv1_bn')(x)
    x = tf.keras.layers.Activation('relu', name='conv1_relu')(x)
    x = tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = tf.keras.layers.MaxPooling2D(3, strides=2, name='pool1')(x)

    x = dense_block(x, blocks[0], name='conv2')
    x = transition_block(x, 0.5, name='pool2')
    x = dense_block(x, blocks[1], name='conv3')
    x = transition_block(x, 0.5, name='pool3')
    x = dense_block(x, blocks[2], name='conv4')
    x = transition_block(x, 0.5, name='pool4')
    x = dense_block(x, blocks[3], name='conv5')

    x = tf.keras.layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='bn')(x)
    x = tf.keras.layers.Activation('relu', name='relu')(x)

    x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)
    
    x = tf.keras.layers.Dense(classes, activation='softmax', name='fc')(x)

    # Create model
    model = tf.keras.models.Model(img_input, x, name='densenet')

    return model


model = DenseNet(blocks=[6, 12, 24, 16], input_shape=(224, 224, 3))

# Load the weights
model.load_weights("DenseNet-BC-121-32.h5")  
for layer in model.layers:
    layer.trainable = False

x = model.layers[-2].output 



x = Dense(512, activation='relu')(x)
x = Dense(5, activation='softmax')(x)
new_model = tf.keras.Model(inputs=model.input, outputs=x)



new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


new_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_2    │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d_2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_3    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_3… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 7,564,869 (28.86 MB)

 Trainable params: 527,365 (2.01 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [26]:
new_model.fit(train, epochs=30, validation_data=val)

Epoch 1/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 77s 8s/step - accuracy: 0.8560 - loss: 0.4630 - val_accuracy: 0.8724 - val_loss: 0.4589
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 75s 8s/step - accuracy: 0.8788 - loss: 0.4018 - val_accuracy: 0.8802 - val_loss: 0.4032
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 73s 7s/step - accuracy: 0.8927 - loss: 0.3573 - val_accuracy: 0.8880 - val_loss: 0.3537
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 76s 8s/step - accuracy: 0.9182 - loss: 0.2989 - val_accuracy: 0.9193 - val_loss: 0.3121
Epoch 5/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 77s 8s/step - accuracy: 0.9208 - loss: 0.2891 - val_accuracy: 0.9141 - val_loss: 0.2844
Epoch 6/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 78s 8s/step - accuracy: 0.9214 - loss: 0.2689 - val_accuracy: 0.9401 - val_loss: 0.2400
Epoch 7/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 80s 8s/step - accuracy: 0.9360 - loss: 0.2330 - val_accuracy: 0.9141 - val_loss: 0.2716
Epoch 8/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 81s 8s/step - accuracy: 0.9154 - loss: 0.2655 - val_accuracy: 0.8984 - val_loss:

In [27]:
from tensorflow.keras.metrics import Precision, Recall, SparseCategoricalAccuracy
pre = Precision()
re = Recall()
acc = SparseCategoricalAccuracy()

In [28]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = new_model.predict(X)
    yhat_classes = np.argmax(yhat, axis=-1)
    
    
    pre.update_state(y, yhat_classes)
    re.update_state(y, yhat_classes)
    acc.update_state(y, yhat_classes)


4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 859ms/step


In [29]:
print(pre.result(), re.result(), acc.result())

tf.Tensor(0.99698794, shape=(), dtype=float32) tf.Tensor(0.99101794, shape=(), dtype=float32) tf.Tensor(0.08715596, shape=(), dtype=float32)


In [30]:
predictions = []
def get_pred(yhat):
    predictions.append(np.argmax(yhat))


In [31]:
import cv2
import os


path = 'Test/'

images = os.listdir(path)
# print(images)
for image in images:
    full_path = path + image
    img = cv2.imread(full_path)
#     print(img)
    img = img/255
    resized_img = tf.image.resize(img, (224, 224))
    yhat = new_model.predict(np.expand_dims(resized_img, 0))
    get_pred(yhat)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━

In [32]:
for i in range(len(images)):
    images[i] = images[i][:-4]

In [33]:
print(len(predictions))
print(len(images))

150
150


In [34]:
import pandas as pd

df = pd.DataFrame({'ID' : images, 'label': predictions})

In [35]:
df.head()

,ID,label
0,image1,0
1,image10,3
2,image100,2
3,image101,3
4,image102,3


In [36]:
df.to_csv('Dense_Net_predictions.csv', sep=',', index=False, encoding='utf-8')
